In [39]:
import pandas as pd 
import sklearn as sk
import numpy as np
import talib
from talib import abstract
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
from xgboost import XGBRegressor, XGBClassifier
# import tuneta
# import pandas_ta as ta
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
# import lightgbm as lgb
import os

# from sklearn.gaussian_process import GaussianProcessRegressor

from scipy.stats import linregress

In [28]:
result = pd.read_csv('./result.csv')
result['date'] = pd.to_datetime(result['date'])

In [51]:
result

,date,stock_id,Unnamed: 0_x,open,close,high,low,volume,amount,ht_dcperiod,...,buy_price,sell_price,lowest_price,highest_price,buy_date,sell_date,profit(%),MAE,BMFE,GMFE
0,2014-12-31,1476,179.0,319.50,321.00,322.50,318.00,428829.0,1.375469e+08,21.413027,...,317.0,311.50,299.0,317.00,2015-01-05,2015-01-09,-1.735016,5.678233,0.000000,0.000000
1,2015-01-09,1476,719.0,314.00,311.50,315.00,309.50,1301640.0,4.051355e+08,18.504793,...,306.5,323.00,306.5,349.50,2015-01-12,2015-02-11,5.383361,0.000000,2.446982,14.029364
2,2015-02-10,1476,539.0,323.00,320.00,328.00,319.00,1253076.0,4.030960e+08,19.770761,...,323.0,385.00,323.0,388.50,2015-02-11,2015-03-11,19.195046,0.000000,0.000000,20.278638
3,2015-03-10,1476,539.0,383.00,388.50,396.50,383.00,1570806.0,6.137458e+08,20.365226,...,385.0,413.50,371.5,413.50,2015-03-11,2015-04-01,7.402597,3.506494,2.337662,7.402597
4,2015-03-31,1476,539.0,411.00,411.50,416.00,407.00,1877617.0,7.744404e+08,28.202077,...,413.5,390.00,390.0,422.50,2015-04-01,2015-04-10,-5.683192,5.683192,2.176542,2.176542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12007,2024-01-10,2739,8279.0,62.50,61.90,62.80,61.10,1351803.0,8.372010e+07,22.200905,...,63.5,58.70,54.9,63.50,2024-01-11,2024-02-05,-7.559055,13.543307,0.000000,0.000000
12008,2024-02-05,2365,5219.0,18.75,19.85,19.85,18.60,66463235.0,1.291956e+09,16.251551,...,21.8,19.90,19.9,25.45,2024-02-15,2024-03-11,-8.715596,8.715596,16.743119,16.743119
12009,2024-03-08,2365,4319.0,23.55,20.75,23.80,20.75,21497379.0,4.589064e+08,21.431088,...,19.9,21.85,19.9,21.85,2024-03-11,2024-03-12,9.798995,0.000000,0.000000,9.798995
12010,2024-03-08,3312,10259.0,46.70,41.55,47.95,41.55,15353749.0,6.731710e+08,37.729504,...,38.2,38.10,38.1,38.20,2024-03-11,2024-03-12,-0.261780,0.261780,0.000000,0.000000


In [29]:
pattern = [name for name in list(result.columns) if name[:3] == 'cdl']

pattern2drop = \
    ['cdl3starsinsouth', 'cdlabandonedbaby' , 'cdlconcealbabyswall'] + \
    ['cdlbreakaway' , 'cdl2crows', 'cdl3blackcrows','cdleveningdojistar' , 'cdleveningdojistar' ] + \
    ['cdl3linestrike' , 'cdl3whitesoldiers' ]

pattern = list(set(pattern) - set(pattern2drop))

In [52]:
pattern

['cdlthrusting',
 'cdlmarubozu',
 'cdlmorningdojistar',
 'cdlrickshawman',
 'cdlcounterattack',
 'cdlgravestonedoji',
 'cdlinneck',
 'cdlhangingman',
 'cdlonneck',
 'cdlpiercing',
 'cdladvanceblock',
 'cdlupsidegap2crows',
 'cdlmathold',
 'cdltristar',
 'cdl3inside',
 'cdlsticksandwich',
 'cdlharami',
 'cdlinvertedhammer',
 'cdllongleggeddoji',
 'cdlhomingpigeon',
 'cdlmatchinglow',
 'cdlspinningtop',
 'cdldoji',
 'cdlhighwave',
 'cdlbelthold',
 'cdlladderbottom',
 'cdllongline',
 'cdlhikkake',
 'cdlkicking',
 'cdlrisefall3methods',
 'cdltasukigap',
 'cdlxsidegap3methods',
 'cdlengulfing',
 'cdlmorningstar',
 'cdl3outside',
 'cdlhammer',
 'cdlhikkakemod',
 'cdlclosingmarubozu',
 'cdlgapsidesidewhite',
 'cdldragonflydoji',
 'cdlshootingstar',
 'cdltakuri',
 'cdlstalledpattern',
 'cdlidentical3crows',
 'cdlunique3river',
 'cdldarkcloudcover',
 'cdlseparatinglines',
 'cdlharamicross',
 'cdlkickingbylength',
 'cdlshortline',
 'cdleveningstar',
 'cdldojistar']

In [30]:
# drop some columns
# drop_columns = ["Unnamed: 0_x","open","close","high","low", "stock_id","acos","asin","profit(%)","buy_price","buy_date", "sell_date","volume","amount","cosh","exp","sinh","Unnamed: 0_y","sell_price","sell_price","lowest_price","highest_price","MAE","BMFE","GMFE"]

date = ['date']  # used for train test split, drop later

momentum = ['aroonup', 'aroondown', 'aroonosc', 'mom']  # momentum indicators, Josh

overlap = ['middleband' ,'midpoint' ,'midprice']  # overlap studies, Josh

volume = []  # volume indicators, regressor

volatility = ['natr']

price_transform = ['wclprice']

cycle = ['ht_dcphase']



In [82]:
# test = result.copy()
# X = test.copy().drop(columns=["Unnamed: 0_x","open","close","high","low", "stock_id","acos","asin","profit(%)","buy_price","buy_date", "sell_date","volume","amount","cosh","exp","sinh","Unnamed: 0_y","sell_price","sell_price","lowest_price","highest_price","MAE","BMFE","GMFE"])

# keep_columns = pattern 
keep_columns = pattern + momentum + overlap + volatility + price_transform

# X = test[keep_columns].copy()
# y = np.array(test[["profit(%)"]])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

X = result[date + keep_columns]
# X = test.copy().drop(columns=["Unnamed: 0_x","open","close","high","low", "stock_id","acos","asin","profit(%)","buy_price","buy_date", "sell_date","volume","amount","cosh","exp","sinh","Unnamed: 0_y","sell_price","sell_price","lowest_price","highest_price","MAE","BMFE","GMFE"])


X_train = np.array(X[X['date'].dt.year <= 2020].drop(columns=["date"]))
X_test = np.array(X[X['date'].dt.year >= 2021].drop(columns=["date"]))
y_train = np.array(test[test['date'].dt.year <= 2020][["profit(%)"]]).reshape(-1)
y_test = np.array(test[test['date'].dt.year >= 2021][["profit(%)"]]).reshape(-1)

# Preprocess

In [44]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Model

In [84]:
from sklearn import metrics
def test(y_true, y_pred):
    print(f'r2 \t {metrics.r2_score(y_true, y_pred)}')
    print(f'corr \t {np.corrcoef(y_true.reshape(-1), y_pred.reshape(-1))[0, 1]}')
    print(f'mae \t {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'rmse \t {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')

## No prediction

In [85]:
y_pred_naive = y_train.mean(axis=0, keepdims=True)
test(y_test, y_pred_naive * np.ones_like(y_test))

r2 	 -0.0011917004581476487
corr 	 -1.0899195180412802e-17
mae 	 8.274999354506045
rmse 	 12.304691203145364


## Linear Regression

In [86]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)
print('train')
test(y_train, lr.predict(X_train))
print('test')
test(y_test, lr.predict(X_test))

train
r2 	 0.017019187677449032
corr 	 0.130471628981536
mae 	 7.306031601427657
rmse 	 10.479370208807753
test
r2 	 -487871446203144.75
corr 	 -0.003670118750193438
mae 	 8558478.932450539
rmse 	 271621910.2459922


In [87]:
linregress(lr.predict(X_test), y_test)

LinregressResult(slope=-1.6624290047155163e-10, intercept=1.3704825174264026, rvalue=-0.0036701187501934377, pvalue=0.8158490759908891, stderr=7.137880144841341e-10, intercept_stderr=0.19388046400346604)

## XGBoost Regressor

In [88]:
from xgboost import XGBRegressor
xgb_regressor = XGBRegressor().fit(X_train, y_train)
test(y_train, xgb_regressor.predict(X_train))
test(y_test, xgb_regressor.predict(X_test))

r2 	 0.6096436265429391
corr 	 0.8247925978534733
mae 	 4.779882436435389
rmse 	 6.603790697560076
r2 	 -0.1757392182543578
corr 	 -0.002544549597632013
mae 	 9.314534530124162
rmse 	 13.334219539118179


## RF

In [89]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor().fit(X_train, y_train)
test(y_train, rf.predict(X_train))
test(y_test, rf.predict(X_test))

r2 	 0.8508028441729467
corr 	 0.9792762469210555
mae 	 2.8417628125044563
rmse 	 4.082657378528845
r2 	 -0.0603725138665947
corr 	 0.030278520802913822
mae 	 8.739009279691553
rmse 	 12.663137706239583


In [90]:
linregress(rf.predict(X_test), y_test)

LinregressResult(slope=0.10935162108664373, intercept=1.2485474963418635, rvalue=0.030278520802913822, pvalue=0.05463655537734588, stderr=0.056885369919754875, intercept_stderr=0.2035892389504938)

# Model save and Load

In [97]:
import pickle


def model_save(column_names, model, save_path):
    """
    Input: 
        column_names: list of feature names used in the model
        model: a (scikit-learn) model that takes np.array as input, and output np.array
    """
    with open(save_path, 'wb') as f:
        pickle.dump({'column_names': column_names, 'model': model}, f)
        

def model_load(load_path):
    with open(load_path, 'rb') as f:
        dic = pickle.load(f)
    
    column_names, model = dic['column_names'], dic['model']
    
    def inference(dataframe):
        X = np.array(dataframe[column_names])
        y = model.predict(X)
        assert len(y) == len(X)
        return y
        
    return inference

In [99]:
model_save(keep_columns, rf, './tmp_model.pkl')

In [100]:
inference = model_load('./tmp_model.pkl')

In [106]:
# how to use "inference"? 
result.head()

,date,stock_id,Unnamed: 0_x,open,close,high,low,volume,amount,ht_dcperiod,...,buy_price,sell_price,lowest_price,highest_price,buy_date,sell_date,profit(%),MAE,BMFE,GMFE
0,2014-12-31,1476,179.0,319.5,321.0,322.5,318.0,428829.0,137546863.0,21.413027,...,317.0,311.5,299.0,317.0,2015-01-05,2015-01-09,-1.735016,5.678233,0.000000,0.000000
1,2015-01-09,1476,719.0,314.0,311.5,315.0,309.5,1301640.0,405135540.0,18.504793,...,306.5,323.0,306.5,349.5,2015-01-12,2015-02-11,5.383361,0.000000,2.446982,14.029364
2,2015-02-10,1476,539.0,323.0,320.0,328.0,319.0,1253076.0,403095972.0,19.770761,...,323.0,385.0,323.0,388.5,2015-02-11,2015-03-11,19.195046,0.000000,0.000000,20.278638
3,2015-03-10,1476,539.0,383.0,388.5,396.5,383.0,1570806.0,613745817.0,20.365226,...,385.0,413.5,371.5,413.5,2015-03-11,2015-04-01,7.402597,3.506494,2.337662,7.402597
4,2015-03-31,1476,539.0,411.0,411.5,416.0,407.0,1877617.0,774440353.0,28.202077,...,413.5,390.0,390.0,422.5,2015-04-01,2015-04-10,-5.683192,5.683192,2.176542,2.176542


In [107]:
inference(result.head())

array([-2.35845419,  3.69011332, 13.25212681,  6.85135278, -3.50908429])